In [14]:
import os
import numpy as np
import moviepy.editor as mp
from googletrans import Translator
from gtts import gTTS
import subprocess
from TTS.api import TTS
import torch
from IPython.display import Audio, display
import whisper
import librosa
import soundfile as sf


rutaVideo = "C://Users//R20//Desktop//samples//boxer.mp4"
rutaAudioIngles = "C://Users//R20//Desktop//samples//output_audio.wav"
rutaAudioGenerado =  "C://Users//R20//Desktop//samples//output_synth.wav"
rutaAudioGeneradoNuevaVelocidad = "C://Users//R20//Desktop//samples//output_synth_adjusted.wav"

model = whisper.load_model("base")
result = model.transcribe(rutaVideo)

whisper_text = result["text"]
whisper_language = result['language']

print("Audio text:", whisper_text)

target_language = "Spanish" #@param ["English", "Spanish", "French", "German", "Italian", "Portuguese", "Polish", "Turkish", "Russian", "Dutch", "Czech", "Arabic", "Chinese (Simplified)"]

# Mapping between full names and ISO 639-1 codes
language_mapping = {
    'English': 'en',
    'Spanish': 'es',
    'Chinese (Simplified)': 'zh-cn'
}

target_language_code = language_mapping[target_language]
from googletrans import Translator
translator = Translator()
translated_text = translator.translate(whisper_text, src=whisper_language, dest=target_language_code).text
print("Translated text:", translated_text)



Audio text:  Because you're a boxer, do you find people want to have a street fight with you? This is a question I get asked all the time, but would you want to have a street fight with a boxer? Definitely not. That's what me not. People think because you're a boxer or a fighter, people want to test themselves against you. For all my life, I've never had anyone see or eat a boxer, I want to have a street fight with him. But I'll be absolutely idiots to want to fight with a boxer because the chances are they'll get knocked straight out.
Translated text: Debido a que eres boxeador, ¿encuentras que la gente quiere tener una pelea callejera contigo?Esta es una pregunta que me hacen todo el tiempo, pero ¿te gustaría tener una pelea callejera con un boxeador?Definitivamente no.Eso es lo que yo no.La gente piensa que porque eres boxeador o luchador, la gente quiere probarse contra ti.A pesar de toda mi vida, nunca he hecho que nadie vea o comiera un boxeador, quiero tener una pelea callejera 

In [ ]:
# Confirmar la licencia
confirmation = input("You must confirm the licensing terms. Do you agree? (y/n): ")
if confirmation.lower() != 'y':
    print("Exiting. License agreement not accepted.")
    exit()

# Crear instancia de TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to("cuda")

video = mp.VideoFileClip(rutaVideo)
audio_clip = video.audio
audio_clip.write_audiofile(rutaAudioIngles)

# Generar audio
tts.tts_to_file(
    translated_text,
    speaker_wav=rutaAudioIngles,
    file_path=rutaAudioGenerado,
    language=target_language_code
)

In [ ]:
from moviepy.editor import TextClip
import librosa

rutaVideo = "C://Users//R20//Desktop//samples//boxer.mp4"
rutaAudioIngles = "C://Users//R20//Desktop//samples//output_audio.wav"
rutaAudioGenerado =  "C://Users//R20//Desktop//samples//output_synth.wav"
rutaAudioGeneradoNuevaVelocidad = "C://Users//R20//Desktop//samples//output_synth_adjusted.wav"

y_original, sr_original = librosa.load(rutaAudioIngles)
y_generated, sr_generated = librosa.load(rutaAudioGenerado)
speed_ratio = 1.5#sr_generated / sr_original
y_adjusted = librosa.effects.time_stretch(y_generated, rate=speed_ratio)
sf.write(rutaAudioGeneradoNuevaVelocidad, y_adjusted, sr_original)

# Mostrar el audio
audio_widget = Audio(filename=rutaAudioGeneradoNuevaVelocidad, autoplay=False)
display(audio_widget)


video = mp.VideoFileClip(rutaVideo)

subtitle = TextClip(translated_text, fontsize=30, color='white', bg_color='black')
subtitle_track = subtitle.set_pos('bottom').set_duration(video.duration)

# Cargar el audio en español
audio_spanish = mp.AudioFileClip(rutaAudioGeneradoNuevaVelocidad)

# Sincronizar el audio en español con el video en inglés
video_with_spanish_audio = video.set_audio(audio_spanish).overlay(subtitle_track, position = (0,0))

# Guardar el video con el audio en español
video_with_spanish_audio.write_videofile("C://Users//R20//Desktop//samples//output_video_spanish.mp4")

In [13]:
from moviepy.editor import CompositeVideoClip
import librosa
import soundfile as sf
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip
import re
from PIL import ImageFont

rutaVideo = "C://Users//R20//Desktop//samples//boxer.mp4"
rutaAudioIngles = "C://Users//R20//Desktop//samples//output_audio.wav"
rutaAudioGenerado =  "C://Users//R20//Desktop//samples//output_synth.wav"
rutaAudioGeneradoNuevaVelocidad = "C://Users//R20//Desktop//samples//output_synth_adjusted.wav"


translated_text = translated_text.replace('.', '. ').replace('?', '? ')


y_original, sr_original = librosa.load(rutaAudioIngles)
y_generated, sr_generated = librosa.load(rutaAudioGenerado)
speed_ratio = 1.5#sr_generated / sr_original
sr_adjusted = int(sr_original * speed_ratio) 
y_adjusted = librosa.effects.time_stretch(y_generated, rate=speed_ratio)
sf.write(rutaAudioGeneradoNuevaVelocidad, y_adjusted, sr_original)

# Mostrar el audio
audio_widget = Audio(filename=rutaAudioGeneradoNuevaVelocidad, autoplay=False)
display(audio_widget)

video = mp.VideoFileClip(rutaVideo)

# Cargar el audio en español
audio_spanish = mp.AudioFileClip(rutaAudioGeneradoNuevaVelocidad)

# Split the translated text into words
words = translated_text.split()

# Group the words into pairs
#subtitle_lines = [' '.join(words[i:i+2]) for i in range(0, len(words), 2)]
subtitle_lines = []
for i in range(0, len(words), 2):
    if len(words[i]) > 8:
        subtitle_lines.append(words[i])
        subtitle_lines.append(words[i+1])
    else:
        subtitle_lines.append(' '.join([words[i], words[i+1]]))



# Remove any empty strings from the list of subtitle lines
subtitle_lines = [line.strip() for line in subtitle_lines if line.strip()]

# Create a list to store the text clips
subtitle_clips = []

# Set the font size and duration for each subtitle line
fontsize = 35
# Register the font
#font = ImageFont.truetype("Bubblegum.ttf", fontsize)
duration = video.duration / len(subtitle_lines)

# Initialize a variable to keep track of the current line number
line_number = 0

# Create a TextClip for each subtitle line and add it to the list
for i, line in enumerate(subtitle_lines):
    line = line.upper()
    if line_number % 3 == 0:
        color = 'red'
    elif line_number % 3 == 1:
        color = 'yellow'
    else:
        color = 'white'

    subtitle = TextClip(line, fontsize=fontsize,font="Super Disco Personal Use.ttf", color=color, bg_color='black')
    subtitle_clip = subtitle.set_pos('center').set_duration(duration).set_start(i * duration)
    subtitle_clips.append(subtitle_clip)

    # Increment the line number
    line_number += 1

# Overlay the subtitle clips on the video clip
video_with_subtitles = CompositeVideoClip([video] + subtitle_clips)

# Set the audio of the composite clip
video_with_subtitles.audio = audio_spanish

# Save the resulting clip
video_with_subtitles.write_videofile("C://Users//R20//Desktop//samples//output_video_spanish.mp4")

Moviepy - Building video C://Users//R20//Desktop//samples//output_video_spanish.mp4.
MoviePy - Writing audio in output_video_spanishTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video C://Users//R20//Desktop//samples//output_video_spanish.mp4



Moviepy - Done !
Moviepy - video ready C://Users//R20//Desktop//samples//output_video_spanish.mp4
